In [79]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [80]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [81]:
## Preprocessing data
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [82]:
## Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [83]:
## Onehot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
one_hot_encode_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoder = one_hot_encode_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns = one_hot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [84]:
## Combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography',axis =1),geo_encoded_df],axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [85]:
## Devide the dataset into independent and dependent features
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

## Split the data in training and testing set
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## SCale these features
scalar_reg = StandardScaler()
x_train = scalar_reg.fit_transform(x_train)
x_test = scalar_reg.transform(x_test)

In [86]:
## save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encode_geo.pkl','wb') as file:
    pickle.dump(one_hot_encode_geo,file)

with open('scalar_reg.pkl','wb') as file:
    pickle.dump(scalar_reg,file)

In [87]:
## ANN regression problem statement

In [88]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [89]:
## Build our ANN model
model = Sequential([
    Dense(64,activation = 'relu', input_shape = (x_train.shape[1],)), ## HL1 connected with input layer and so input shape is given
    Dense(32,activation = 'relu'), ## HL2
    Dense(1) ## output # if we do't mention any activation function in output, it will take linear activation function by default for regression
])

In [90]:
## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [91]:
## Set up the tensorboard

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [92]:
## Set up Early stopping - We can train our model in many epocs but to know in which epoch it can give best output, we use this
early_stopping_callback = EarlyStopping(monitor='val_loss',patience = 20, restore_best_weights = True) 

In [93]:
## Training the model
histoy = model.fit(
    x_train,y_train,validation_data = (x_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 100363.2734 - mae: 100363.2734 - val_loss: 98466.3047 - val_mae: 98466.3047
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99456.7812 - mae: 99456.7812 - val_loss: 96637.2734 - val_mae: 96637.2734
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 96261.9062 - mae: 96261.9062 - val_loss: 91943.1797 - val_mae: 91943.1797
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 90007.8047 - mae: 90007.8047 - val_loss: 84252.5781 - val_mae: 84252.5781
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 81150.8828 - mae: 81150.8828 - val_loss: 74742.6250 - val_mae: 74742.6250
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 71234.7500 - mae: 71234.7500 - val_loss: 65401.8320 - val_mae: 65401.8320
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 62238.8281 - mae: 62238.8281 

In [94]:
## Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [95]:
%tensorboard --logdir regression_logs/fit

Reusing TensorBoard on port 6006 (pid 36104), started 0:47:28 ago. (Use '!kill 36104' to kill it.)

In [96]:
## Evaluate model on test data
test_loss,test_mae = model.evaluate(x_test,y_test)
print(f"Test MAE : {test_mae}")

63/63 [==============================] - 0s 4ms/step - loss: 50290.5000 - mae: 50290.5000
Test MAE : 50290.5


In [97]:
model.save('regression_model.h5')

c:\Users\Dell\Desktop\data\science\ds\ANN classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
